In [ ]:
try:
    # settings colab:
    import google.colab
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


## Dataset

El dataset tiene información sobre propiedades en Boston. El objetivo es predecir el precio de estas propiedades.

https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

http://jse.amstat.org/v19n3/decock.pdf

Para ver la descripción de los campos que componen este dataset:
<a href="../Data/data_description.txt">data_description</a>

## Problema

Transformando las variables del dataset de propiedades en Boston usando StandardScaler y OneHotEncoding, vamos a estimar los valores del campo `SalePrice` usando modelos de regresión lineal.

Después, vamos a comparar los resultados de las predicciones de estos modelos con los que obtenemos usando como variables explicativas la proyección en las componentes principales del dataset transformado.


## Imports

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

## Ejercicio 1 - Carga de datos

In [4]:
df = pd.read_csv('Data/emision-co2-autos_limpio.csv')


In [5]:
df.head()

,vehiculo_marca,vehiculo_modelo,vehiculo_tipo,vehiculo_traccion,vehiculo_id_motor,vehiculo_cilindrada,vehiculo_potencia,vehiculo_tipo_transmision,vehiculo_tipo_combustible,vehiculo_standard_emision,lca_numero,fecha_firma,ensayo_gei_numero,ensayo_gei_laboratorio,emision_CO2,consumo_urbano,consumo_extraurbano,consumo_mixto,id_etiqueta
0,TOYOTA,LAND CRUISER 200,SUV,4x4,TOYOTA 1VD-FTV,4461.0,NaN,AUTOMATICA,GAS OIL,EURO V,NaN,04/10/2017,H1860666086/241,VINÇOTTE nv,260.70,11.56,8.94,9.90,000001A
1,RENAULT,FLUENCE 2.0 16V,SEDÁN 4 PUERTAS,4x2,RENAULT M4RK7,1997.0,NaN,CVT,NAFTA,EURO V,NaN,22/06/2016,09/09790,UTAC,175.40,10.50,6.10,7.70,000178A
2,RENAULT,DUSTER 2.0 16v,SEDÁN 5 PUERTAS,4x2,RENAULT F4R E4,1998.0,105,MANUAL,NAFTA,EURO V,NaN,NaN,R1-0210/17,DELPHI,198.86,11.13,6.98,8.52,000650C
3,RENAULT,DUSTER 2.0 16v 4X4,SEDÁN 5 PUERTAS,4x4,RENAULT F4R E4,1998.0,105,MANUAL,NAFTA,EURO V,NaN,NaN,R1-0209/17,DELPHI,199.74,11.20,7.01,8.55,000659C
4,CITROËN,DS4,COUPÉ 3 + 2 PUERTAS,4x2,CITROËN EP6CDTM (5FM),1598.0,NaN,AUTOMATICA,NAFTA,EURO V,NaN,11/10/2011,11/04511,UTAC,177.60,10.60,6.00,7.70,000106A


In [ ]:
data.dtypes

In [6]:
df.drop(['vehiculo_potencia','lca_numero', 'vehiculo_id_motor','vehiculo_id_motor','fecha_firma','ensayo_gei_numero','ensayo_gei_laboratorio','id_etiqueta'], axis=1, inplace=True)

In [9]:
df.head()

,vehiculo_marca,vehiculo_modelo,vehiculo_tipo,vehiculo_traccion,vehiculo_cilindrada,vehiculo_tipo_transmision,vehiculo_tipo_combustible,vehiculo_standard_emision,emision_CO2,consumo_urbano,consumo_extraurbano,consumo_mixto
0,TOYOTA,LAND CRUISER 200,SUV,4x4,4461.0,AUTOMATICA,GAS OIL,EURO V,260.70,11.56,8.94,9.90
1,RENAULT,FLUENCE 2.0 16V,SEDÁN 4 PUERTAS,4x2,1997.0,CVT,NAFTA,EURO V,175.40,10.50,6.10,7.70
2,RENAULT,DUSTER 2.0 16v,SEDÁN 5 PUERTAS,4x2,1998.0,MANUAL,NAFTA,EURO V,198.86,11.13,6.98,8.52
3,RENAULT,DUSTER 2.0 16v 4X4,SEDÁN 5 PUERTAS,4x4,1998.0,MANUAL,NAFTA,EURO V,199.74,11.20,7.01,8.55
4,CITROËN,DS4,COUPÉ 3 + 2 PUERTAS,4x2,1598.0,AUTOMATICA,NAFTA,EURO V,177.60,10.60,6.00,7.70


In [10]:
df.dropna(inplace=True)

In [11]:
df.isna().sum()

vehiculo_marca               0
vehiculo_modelo              0
vehiculo_tipo                0
vehiculo_traccion            0
vehiculo_cilindrada          0
vehiculo_tipo_transmision    0
vehiculo_tipo_combustible    0
vehiculo_standard_emision    0
emision_CO2                  0
consumo_urbano               0
consumo_extraurbano          0
consumo_mixto                0
dtype: int64

In [12]:
df.dtypes

vehiculo_marca                object
vehiculo_modelo               object
vehiculo_tipo                 object
vehiculo_traccion             object
vehiculo_cilindrada          float64
vehiculo_tipo_transmision     object
vehiculo_tipo_combustible     object
vehiculo_standard_emision     object
emision_CO2                  float64
consumo_urbano               float64
consumo_extraurbano          float64
consumo_mixto                float64
dtype: object

In [14]:
target = df["emision_CO2"]

In [15]:
df['vehiculo_traccion'].value_counts()

4x2          286
4x4           95
4x2 - 4x4     25
Name: vehiculo_traccion, dtype: int64

In [16]:
df['vehiculo_tipo_transmision'].value_counts()

AUTOMATICA                                     223
MANUAL                                         147
CVT                                             33
CTT                                              1
E-CVT (Hibrida controlada electrónicamente)      1
MANUAL AUTOMATIZADA                              1
Name: vehiculo_tipo_transmision, dtype: int64

In [21]:
target

0      260.70
1      175.40
2      198.86
3      199.74
4      177.60
        ...  
407     99.55
408    120.00
409    186.90
410    163.10
411    192.83
Name: emision_CO2, Length: 406, dtype: float64

## Ejercicio 2 - Categorical y Non-categorical 

**2.1** Determinemos qué variables que forman el dataset son categóricas. Consideremos categóricas variables numéricas que tienen como máximo cinco valores distintos.

**2.2** Usemos `pandas.Series.astype` para convertir esas nuevas columnas categóricas en `numpy.object`

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Categorical.html


In [17]:
df.columns

Index(['vehiculo_marca', 'vehiculo_modelo', 'vehiculo_tipo',
       'vehiculo_traccion', 'vehiculo_cilindrada', 'vehiculo_tipo_transmision',
       'vehiculo_tipo_combustible', 'vehiculo_standard_emision', 'emision_CO2',
       'consumo_urbano', 'consumo_extraurbano', 'consumo_mixto'],
      dtype='object')

In [31]:
categorical_columns = ['vehiculo_marca', 'vehiculo_traccion', 'vehiculo_tipo_transmision', 'vehiculo_standard_emision',
       'vehiculo_tipo_combustible']
categorical_columns

['vehiculo_marca',
 'vehiculo_traccion',
 'vehiculo_tipo_transmision',
 'vehiculo_standard_emision',
 'vehiculo_tipo_combustible']

In [32]:
non_categorical_columns = ['vehiculo_cilindrada', 'consumo_urbano', 'consumo_extraurbano', 'consumo_mixto']
non_categorical_columns



['vehiculo_cilindrada',
 'consumo_urbano',
 'consumo_extraurbano',
 'consumo_mixto']

## Ejercicio 3 - Valores nulos

Calculemos el porcentaje de nulos en cada columna.

Eliminemos aquellas columnas que tengan al menos el 40% de los registros con valores nulos.

Eliminemos aquellos registros que tengan al menos un valor nulo en las columnas que no fueron eliminadas en el paso anterior.

Eliminemos la columna `Id` del dataset

In [26]:
df.head()

,vehiculo_marca,vehiculo_modelo,vehiculo_tipo,vehiculo_traccion,vehiculo_cilindrada,vehiculo_tipo_transmision,vehiculo_tipo_combustible,vehiculo_standard_emision,emision_CO2,consumo_urbano,consumo_extraurbano,consumo_mixto
0,TOYOTA,LAND CRUISER 200,SUV,4x4,4461.0,AUTOMATICA,GAS OIL,EURO V,260.70,11.56,8.94,9.90
1,RENAULT,FLUENCE 2.0 16V,SEDÁN 4 PUERTAS,4x2,1997.0,CVT,NAFTA,EURO V,175.40,10.50,6.10,7.70
2,RENAULT,DUSTER 2.0 16v,SEDÁN 5 PUERTAS,4x2,1998.0,MANUAL,NAFTA,EURO V,198.86,11.13,6.98,8.52
3,RENAULT,DUSTER 2.0 16v 4X4,SEDÁN 5 PUERTAS,4x4,1998.0,MANUAL,NAFTA,EURO V,199.74,11.20,7.01,8.55
4,CITROËN,DS4,COUPÉ 3 + 2 PUERTAS,4x2,1598.0,AUTOMATICA,NAFTA,EURO V,177.60,10.60,6.00,7.70


## Ejercicio 4 - Train test sets

Dividamos el dataset de registros completos en 70-30 train test, definiendo como X_train, X_test las matrices de features e Y_train, Y_test los vectores target.


In [28]:
X = df.drop(['vehiculo_modelo', 'vehiculo_tipo', 'emision_CO2'], axis = 1)

In [29]:
Y = target

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 1237)

## Ejercicio 5 - Feature engineering

**5.1** Usemos one hot encoding para transformar las variables categóricas de los dataset de train y test

**5.2** Usemos StandardScaler para transformar las variables numéricas de los dataset de train y test

In [36]:
encoder_categories = []

for col in categorical_columns:    
    col_categories = df[col].unique()
    encoder_categories.append(col_categories)

encoder_categories

[array(['TOYOTA', 'RENAULT', 'CITROËN', 'LEXUS', 'PEUGEOT', 'FORD',
        'PEUGEOT / CITROËN', 'HYUNDAI', 'SUZUKI', 'CHEVROLET', 'LIFAN',
        'BMW', 'CITROËN / DS', 'HONDA', 'NISSAN', 'FIAT', 'CHERY',
        'VOLKSWAGEN', 'ALFA ROMEO', 'FAW', 'MERCEDES-BENZ', 'JEEP',
        'LAND ROVER', 'JAC', 'VOLVO', 'SHINERAY', 'CHANGAN', 'DS', 'MINI',
        'DOMY', 'BORGWARD', 'PORSCHE', 'AUDI', 'BAIC', 'ZANELLA', 'RAM',
        'FOTON', 'MASERATI', 'SUBARU', 'HAVAL', 'ISUZU', 'DONGFENG',
        'MERCEDES-AMG', 'MITSUBISHI', 'ZOTYE', 'KIA', 'GREAT WALL', 'KYC',
        'FERRARI'], dtype=object),
 array(['4x4', '4x2', '4x2 - 4x4'], dtype=object),
 array(['AUTOMATICA', 'CVT', 'MANUAL', 'CTT', 'MANUAL AUTOMATIZADA',
        'E-CVT (Hibrida controlada electrónicamente)'], dtype=object),
 array(['EURO V', 'EURO VI', 'EURO VI b', 'EURUO V', 'EURO VI a',
        'EURO  V', 'EURO VI c', 'EURO VI d', 'EURO  VI b'], dtype=object),
 array(['GAS OIL', 'NAFTA', 'NAFTA / ELECTRICIDAD', 'DIESEL'], dty

In [37]:
encoder = OneHotEncoder(categories = encoder_categories, sparse=False)

encoder = encoder.fit(X_train[categorical_columns])
    
X_train_encoded = encoder.transform(X_train[categorical_columns])
X_train_categorical = pd.DataFrame(X_train_encoded, columns = encoder.get_feature_names(categorical_columns))
#X_train_categorical.sample(5)

X_test_encoded = encoder.transform(X_test[categorical_columns])
X_test_categorical = pd.DataFrame(X_test_encoded, columns = encoder.get_feature_names(categorical_columns))
X_test_categorical.head()

,vehiculo_marca_TOYOTA,vehiculo_marca_RENAULT,vehiculo_marca_CITROËN,vehiculo_marca_LEXUS,vehiculo_marca_PEUGEOT,vehiculo_marca_FORD,vehiculo_marca_PEUGEOT / CITROËN,vehiculo_marca_HYUNDAI,vehiculo_marca_SUZUKI,vehiculo_marca_CHEVROLET,...,vehiculo_standard_emision_EURUO V,vehiculo_standard_emision_EURO VI a,vehiculo_standard_emision_EURO V,vehiculo_standard_emision_EURO VI c,vehiculo_standard_emision_EURO VI d,vehiculo_standard_emision_EURO VI b,vehiculo_tipo_combustible_GAS OIL,vehiculo_tipo_combustible_NAFTA,vehiculo_tipo_combustible_NAFTA / ELECTRICIDAD,vehiculo_tipo_combustible_DIESEL
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [38]:
# 5.2
std_sclr = StandardScaler()
std_sclr_trained = std_sclr.fit(X_train[non_categorical_columns])
X_train_numerical = std_sclr_trained.transform(X_train[non_categorical_columns])
X_train_numerical_scaled = pd.DataFrame(X_train_numerical, columns = non_categorical_columns)
X_train_numerical_scaled.head()

,vehiculo_cilindrada,consumo_urbano,consumo_extraurbano,consumo_mixto
0,-0.717819,0.105950,0.863240,-0.031121
1,-0.110137,-0.488142,-1.029077,-0.104737
2,0.456703,-0.002067,0.933326,-0.036198
3,-0.915841,-0.488142,-0.958991,-0.104737
4,-0.092810,-0.434134,-1.169248,-0.104737


In [39]:
X_test_numerical = std_sclr_trained.transform(X_test[non_categorical_columns])
X_test_numerical_scaled = pd.DataFrame(X_test_numerical, columns = non_categorical_columns)
X_test_numerical_scaled.head()

,vehiculo_cilindrada,consumo_urbano,consumo_extraurbano,consumo_mixto
0,0.844084,-0.164092,0.582897,-0.051429
1,-0.917079,-0.344120,-0.755742,-0.091283
2,-0.596530,-0.002067,0.232468,-0.051429
3,1.728998,0.932279,1.416917,0.024471
4,-1.073022,-0.146089,-0.328219,-0.071737


In [ ]:
td_sclr_target = StandardScaler()
std_sclr_target_trained = std_sclr_target.fit(Y_train)
Y_train_numerical = std_sclr_target_trained.transform(Y_train)
#Y_train_numerical

Y_test_numerical = std_sclr_target_trained.transform(Y_test)
#Y_test_numerical

In [ ]:
X_train_transf = pd.concat([X_train_categorical, X_train_numerical_scaled], axis = 1)
print(X_train_categorical.shape)
print(X_train_numerical_scaled.shape)
print(X_train_transf.shape)

In [ ]:
X_test_transf = pd.concat([X_test_categorical, X_test_numerical_scaled], axis = 1)
print(X_test_categorical.shape)
print(X_test_numerical_scaled.shape)
print(X_test_transf.shape)


## Ejercicio 6 - Regresión lineal

**6.1** Constuyamos un modelo de regresión lineal múltiple sobre los datasets de train y test transformados.

**6.2** Construyamos otro modelo usando regularización Lasso

**6.2** Construyamos un terecer modelo usando regularización Ridge

Para los tres modelos calculemos:
* mean_absolute_error
* mean_squared_error
* raiz cuadrada de mean_squared_error
* r2

In [ ]:
linreg = linear_model.LinearRegression()
linreg.fit(X_train_transf, Y_train_numerical)
Y_pred = linreg.predict(X_test_transf)

true_values = Y_test_numerical
predicted_values = Y_pred

In [ ]:
reg_metrics = pd.Series([
                metrics.mean_absolute_error(true_values, predicted_values), 
                metrics.mean_squared_error(true_values, predicted_values),
                np.sqrt(metrics.mean_squared_error(true_values, predicted_values)),
                metrics.r2_score(true_values, predicted_values)])


reg_metrics.index = ['Mean Absolute Error:', 'Mean Squared Error:', 'RMSE:', 'R2:']

print(reg_metrics)

Regularización Lasso

In [ ]:
lasso = linear_model.Lasso(alpha=0.1)
lasso.fit(X_train_transf, Y_train_numerical)
Y_pred = lasso.predict(X_test_transf)

true_values = Y_test_numerical
predicted_values = Y_pred

lasso_metrics = pd.Series([
                metrics.mean_absolute_error(true_values, predicted_values), 
                metrics.mean_squared_error(true_values, predicted_values),
                np.sqrt(metrics.mean_squared_error(true_values, predicted_values)),
                metrics.r2_score(true_values, predicted_values)])


lasso_metrics.index = ['Mean Absolute Error:', 'Mean Squared Error:', 'RMSE:', 'R2:']

print(lasso_metrics)

Regularización Ridge

In [ ]:
ridge = linear_model.Ridge(alpha=0.1)
ridge.fit(X_train_transf, Y_train_numerical)
Y_pred = ridge.predict(X_test_transf)

true_values = Y_test_numerical
predicted_values = Y_pred

ridge_metrics = pd.Series([
                metrics.mean_absolute_error(true_values, predicted_values), 
                metrics.mean_squared_error(true_values, predicted_values),
                np.sqrt(metrics.mean_squared_error(true_values, predicted_values)),
                metrics.r2_score(true_values, predicted_values)])


ridge_metrics.index = ['Mean Absolute Error:', 'Mean Squared Error:', 'RMSE:', 'R2:']

print(ridge_metrics)

## Ejercicio 7 - PCA

Grafiquemos la varianza explicada por 100, 50 y 30 componentes principales del dataset de train transformado con one hot encoding y StandardScaler


In [ ]:
def plot_explained_variance(components_count, X):

    model_pca = PCA(components_count).fit(X)

    explained_variance = model_pca.explained_variance_ratio_

    #print(explained_variance)

    cumulative_explained_variance = np.cumsum(explained_variance)

    #print(cumulative_explained_variance)

    plt.plot(cumulative_explained_variance)
    plt.xlabel('número de componentes')
    plt.ylabel('% de varianza explicada');
        

In [ ]:
plot_explained_variance(components_count = 100, X = X_train_transf)

In [ ]:
plot_explained_variance(components_count = 50, X = X_train_transf)

In [ ]:
plot_explained_variance(components_count = 30, X = X_train_transf)

## Ejercicio 8 - Proyección sobre las componentes principales del conjunto de entrenamiento

Construyamos un conjunto de entrenamiento y uno de test para una regresión lineal usando las primeras 30 componentes principales del conjunto de entrenamiento usado en la regresión lineal del ejercicio anterior.

In [ ]:
model_pca = PCA(30).fit(X_train_transf)
X_train_PCA = model_pca.transform(X_train_transf)
X_test_PCA = model_pca.transform(X_test_transf)

## Ejercicio 9 - Regresión lineal sobre componentes principales

**6.1** Constuyamos un modelo de regresión lineal múltiple sobre los datasets de train y test usando como features la proyección sobre las componentes principales calculadas en el ejercicio anterior. ¿Qué supuesto podemos asegurar que se cumple usando las componentes principales como variables explicativas?

**6.2** Construyamos otro modelo usando regularización Lasso sobre los mismos features que en 6.1

**6.2** Construyamos un terecer modelo usando regularización Ridge sobre los mismos features que en 6.1

Para los tres modelos calculemos:
* mean_absolute_error
* mean_squared_error
* raiz cuadrada de mean_squared_error
* r2

In [ ]:
# cada una de las "nuevas" variables después de PCA son independientes entre sí. 
# los supuestos de un modelo lineal requieren que nuestras variables independientes sean independientes entre sí. 
# Si ajustamos un modelo de regresión lineal con estas "nuevas" variables, este supuesto necesariamente se cumplirá.

linreg_pca = linear_model.LinearRegression()
linreg_pca.fit(X_train_PCA, Y_train_numerical)
# linear_fit
Y_pred = linreg_pca.predict(X_test_PCA)

true_values = Y_test_numerical
predicted_values = Y_pred

In [ ]:
reg_pca_metrics = pd.Series([
                metrics.mean_absolute_error(true_values, predicted_values), 
                metrics.mean_squared_error(true_values, predicted_values),
                np.sqrt(metrics.mean_squared_error(true_values, predicted_values)),
                metrics.r2_score(true_values, predicted_values)])


reg_pca_metrics.index = ['Mean Absolute Error:', 'Mean Squared Error:', 'RMSE:', 'R2:']

print(reg_pca_metrics)

In [ ]:
lasso = linear_model.Lasso(alpha=0.1)
lasso.fit(X_train_PCA, Y_train_numerical)
Y_pred = lasso.predict(X_test_PCA)

true_values = Y_test_numerical
predicted_values = Y_pred

lasso_pca_metrics = pd.Series([
                metrics.mean_absolute_error(true_values, predicted_values), 
                metrics.mean_squared_error(true_values, predicted_values),
                np.sqrt(metrics.mean_squared_error(true_values, predicted_values)),
                metrics.r2_score(true_values, predicted_values)])


lasso_pca_metrics.index = ['Mean Absolute Error:', 'Mean Squared Error:', 'RMSE:', 'R2:']

print(lasso_pca_metrics)

In [ ]:
ridge = linear_model.Ridge(alpha=0.1)
ridge.fit(X_train_PCA, Y_train_numerical)
Y_pred = ridge.predict(X_test_PCA)

true_values = Y_test_numerical
predicted_values = Y_pred

ridge_pca_metrics = pd.Series([
                metrics.mean_absolute_error(true_values, predicted_values), 
                metrics.mean_squared_error(true_values, predicted_values),
                np.sqrt(metrics.mean_squared_error(true_values, predicted_values)),
                metrics.r2_score(true_values, predicted_values)])


ridge_pca_metrics.index = ['Mean Absolute Error:', 'Mean Squared Error:', 'RMSE:', 'R2:']

print(ridge_pca_metrics)

In [ ]:
metrics = pd.DataFrame([reg_metrics, lasso_metrics, ridge_metrics, 
                        reg_pca_metrics, lasso_pca_metrics, ridge_pca_metrics ])
metrics.index = ["regresion lineal", "lasso", "ridge", "pca - regresion lineal", "pca - lasso", "pca - ridge"]
metrics

Vemos que, para la regresión lineal múltiple, todas las métricas que obtenemos con el modelo entrenado con las componentes principales son mejores que las del modelo sin esta transformación.

Con regularización de Lasso o Ridge no son tan evidentes las diferencias entre usar o no la proyección sobre las componenetes principales.

Una ventaja de este segundo modelo es que usa 30 features mientras que el primero usa 280.

Como ejercicio adicional, repitan los tres modelos usando en el ejercicio 5 
`encoder = OneHotEncoder(categories = encoder_categories, sparse=False, drop="first")`

Y comparen estos nuevos resultados.

**¿Qué significa que R2 sea negativo?**

## Referencias

https://www.kaggle.com/miguelangelnieto/pca-and-regression